# Web Scraper for *everynoise.com*

In [ ]:
#adapted from https://github.com/aweitz/EveryNoise/blob/master/scrapGenres.ipynb
from bs4 import BeautifulSoup
import requests, sys
import pandas as pd

In [ ]:
r = requests.get("http://everynoise.com/everyschool.cgi?scope=US")

## Scraper for *everyschool*

Goes to the website and loops through all the schools that are in the USA. Gets the top genres of each school. 

In [ ]:
# Link here and read into Python, or just an html link as markdown.
# ?root=The%20University%20of%20Texas%20at%20Austin&
import re
norm = re.compile('[+»&\s-]')
soup = BeautifulSoup(r.text,"html.parser")
allSchools = soup.find_all("tr", "datarow")
## Cycle through genres and go to that genres page
schools = schools[:-1]
# popular_genres = []
# playlists = []
# schCnt = 0
for schoolRow in allSchools[629:]:
    print("Pulling school #" + str(schCnt), end = '\t')
    sys.stdout.flush()
    school = schoolRow.text.split('USA')[1].strip()
    print(school)
    schools.append(school)
    schoolPage = "http://everynoise.com/everyschool.cgi?root=" + school.replace(' ','%20') + "&scope=US"
    ## Pull artists from genre page
    r2 = requests.get(schoolPage)
    soup2 = BeautifulSoup(r2.text,"html.parser")
    schools_genre_div = soup2.find_all("div","note")
    inner = soup2.find_all("div","note")
    for d in inner:
        genres = d.text.split('\n')[1:-1]
        genres = [re.sub(norm,'',g) for g in genres]
    popular_genres.append(genres)
    spotify = soup2.find("iframe")
    playlists.append(spotify['src'])
    schCnt = schCnt+1

## Constructs the DataFrame for the USA Universities

In [ ]:
print(len(schools),len(popular_genres),len(playlists))
df = pd.DataFrame({'SCHOOL':schools, 'GENRES':popular_genres, 'PLAYLIST_LINK':playlists})
df.to_csv('school_genres.csv')
df

## Scraper for *genremap* 
Goes to the website and enters each genre link and scraps the html. Grabs the related genres, opposite genres, and uses the font size for each as a  indicator of edge 'weight'. Also gets the Spotify playlist links.

In [ ]:
r = requests.get("http://everynoise.com/engenremap.html")

In [ ]:
import re

soup = BeautifulSoup(r.text,"html.parser")
allGenreDivs = soup.find_all("div", "genre scanme")

## Cycle through genres and go to that genres page
genreListArtist = []
genreListOpp = []
genreListSim = []
genreList = []
playlists = []
simWeights = []
oppWeights = []
artistWeights = []
genreCnt = 0
for genreDiv in allGenreDivs:
    print("Pulling genre #" + str(genreCnt))
    sys.stdout.flush()
    genre = (genreDiv.text)
    genre = re.sub("[:'+»&\s-]", '', genre)    
    genreList.append(genre)
    genrePage = "http://everynoise.com/engenremap-" + genre + ".html"

    ## Pull artists from genre page
    r2 = requests.get(genrePage)
    soup2 = BeautifulSoup(r2.text,"html.parser")
    spotify_link = soup2.find_all("a", text = 'playlist')
    playlists.append(spotify_link[0]['href'])
    allArtistDivs = set(soup2.find_all("div", "genre scanme"))
    allGenresRelated = set(soup2.find_all("div", "genre"))
    allGenresRelated = allGenresRelated - allArtistDivs
    artistList = []
    art_w = []
    
    for artist in allArtistDivs:
        weight = artist['style'].split()[-1].replace('%', '')
        artistName = artist.text.strip().replace("»", "").replace(' ', '')
        if not(artistName.isspace()):   
            art_w.append(weight)
            artistList.append(artistName) 
    artistWeights.append(art_w)
    genreListArtist.append(artistList)
    
    simGenres = []
    oppGenres = []
    weights_sim = []
    weights_opp = []
    
    for other_genre in allGenresRelated:
        weight = other_genre['style'].split()[-1].replace('%', '')
        if 'nearby' in other_genre['id']:
            weights_sim.append(weight)
            simGenres.append(other_genre.text.strip().replace("»", "").replace(' ', ''))
        elif 'mirror' in other_genre['id']:
            weights_opp.append(weight)
            oppGenres.append(other_genre.text.strip().replace("»", "").replace(' ', ''))
    genreListSim.append(simGenres)
    genreListOpp.append(oppGenres)
    simWeights.append(weights_sim)
    oppWeights.append(weights_opp)
    genreCnt = genreCnt+1

## Constructs the Dataframe for All Genres and Converts it to a CSV

In [ ]:
# genreListArtist = []
# genreListOpp = []
# genreListSim = []
# genreList = []
# playlists = []
df = pd.DataFrame({"GENRE":genreList, 
                   "SIM_GENRES":genreListSim,
                   "SIM_WEIGHTS":simWeights,
                   "OPP_GENRES":genreListOpp,
                   "OPP_WEIGHTS":oppWeights,
                   "REL_ARTISTS":genreListArtist, 
                   "ARTIST_WEIGHTS":artistWeights,
                   "SPOTIFY_URL":playlists})
df.to_csv('all_genres.csv')
df